In [1]:
import sys
from pathlib import Path

# 현재 폴더: ptta/other_method/ActMAD/
# ptta 바로 위의 디렉토리를 sys.path에 추가
PROJECT_PARENT = Path.cwd().parents[0]  # -> ptta/ 의 부모 디렉토리
print(PROJECT_PARENT)
sys.path.insert(0, str(PROJECT_PARENT))

from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection, SHIFTDiscreteSubsetForObjectDetection

# from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

# import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

/workspace/ptta


## Check GPU Availability

In [2]:
!nvidia-smi

Tue Sep  9 03:26:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   40C    P0              34W / 250W |   2038MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0
ADDITIONAL_GPU = 0
DATA_TYPE = torch.bfloat16

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda:0


In [4]:
PROJECT_NAME = "ActMAD test"
RUN_NAME = "RT-DETR_R50_ActMAD"

## Baseline

In [5]:
import baseline

data_root = path.normpath(path.join(Path.cwd(), "..", "data"))
print(data_root)
direct_method = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./direct_method_checkpoints",
    method="direct_method",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=32,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

actmad = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./actmad_checkpoints",
    method="actmad",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

norm = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./norm_checkpoints",
    method="norm",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./dua_checkpoints",
    method="dua",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

mean_teacher = dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./mean_teacher_checkpoints",
    method="mean_teacher",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=4,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    use_scheduler=False,
    warmup_steps=50,
    decay_total_steps=20,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

/workspace/ptta/data


In [6]:
mean_teacher.train()

[2025-09-09 03:26:32,014] INFO: Log file: ./mean_teacher_checkpoints/mean_teacher_20250909_032632.log
[2025-09-09 03:26:32,526] INFO: start cloudy
[2025-09-09 03:26:32,665] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-09-09 03:26:32,667] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[2025-09-09 03:26:32,668] INFO: INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
[2025-09-09 03:26:32,669] INFO: INFO: Downloading 'SHIFT_SUBSET' from file server to /workspace/ptta/data/SHIFT/discrete...
[2025-09-09 03:26:32,670] INFO: INFO: Dataset archive found in the root directory. Skipping download.
[09/09/2025 03:26:32] SHIFT DevKit - INFO - Base: /workspace/ptta/data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f6daf9ccf10>
[09/09/2025 03:26:32] SHIFT DevKit - INFO - Loading annotation from '/workspace/ptta/data/SHIFT_SUBSET/cloudy_daytime/discr

  0%|          | 0/13850 [00:00<?, ?it/s]

[2025-09-09 03:26:59,920] ERROR: UserWarning: This overload of add_ is deprecated:
[2025-09-09 03:26:59,923] ERROR: 	add_(Number alpha, Tensor other)
[2025-09-09 03:26:59,925] ERROR: Consider using one of the following signatures instead:
[2025-09-09 03:26:59,926] ERROR: 	add_(Tensor other, *, Number alpha = 1) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1691.)


  0%|          | 0/600 [00:00<?, ?it/s]

[2025-09-09 03:31:59,103] INFO: mAP@0.95_cloudy: 0.486
[2025-09-09 03:31:59,109] INFO: mAP50_cloudy: 0.663
[2025-09-09 03:31:59,110] INFO: mAP75_cloudy: 0.533
[2025-09-09 03:31:59,111] INFO: pedestrian_mAP@0.95_cloudy: 0.459
[2025-09-09 03:31:59,112] INFO: car_mAP@0.95_cloudy: 0.558
[2025-09-09 03:31:59,113] INFO: truck_mAP@0.95_cloudy: 0.556
[2025-09-09 03:31:59,115] INFO: bus_mAP@0.95_cloudy: 0.458
[2025-09-09 03:31:59,116] INFO: motorcycle_mAP@0.95_cloudy: 0.443
[2025-09-09 03:31:59,116] INFO: bicycle_mAP@0.95_cloudy: 0.443
[2025-09-09 03:31:59,147] INFO: [cloudy] batch 0: mAP50 -1.0000 -> 0.6633 ✔


...

[2025-09-09 07:35:52,720] INFO: mAP@0.95_foggy: 0.221
[2025-09-09 07:35:52,723] INFO: mAP50_foggy: 0.281
[2025-09-09 07:35:52,724] INFO: mAP75_foggy: 0.236
[2025-09-09 07:35:52,725] INFO: pedestrian_mAP@0.95_foggy: 0.277
[2025-09-09 07:35:52,726] INFO: car_mAP@0.95_foggy: 0.213
[2025-09-09 07:35:52,727] INFO: truck_mAP@0.95_foggy: 0.234
[2025-09-09 07:35:52,728] INFO: bus_mAP@0.95_foggy: 0.090
[2025-09-09 07:35:52,729] INFO: motorcycle_mAP@0.95_foggy: 0.250
[2025-09-09 07:35:52,730] INFO: bicycle_mAP@0.95_foggy: 0.263
[2025-09-09 07:35:52,752] INFO: [foggy] batch 27: mAP50 0.2806 (no-imp 5/10)


  0%|          | 0/663 [00:00<?, ?it/s]

[2025-09-09 07:40:46,826] INFO: mAP@0.95_foggy: 0.221
[2025-09-09 07:40:46,832] INFO: mAP50_foggy: 0.281
[2025-09-09 07:40:46,833] INFO: mAP75_foggy: 0.236
[2025-09-09 07:40:46,834] INFO: pedestrian_mAP@0.95_foggy: 0.277
[2025-09-09 07:40:46,835] INFO: car_mAP@0.95_foggy: 0.212
[2025-09-09 07:40:46,836] INFO: truck_mAP@0.95_foggy: 0.233
[2025-09-09 07:40:46,837] INFO: bus_mAP@0.95_foggy: 0.089
[2025-09-09 07:40:46,837] INFO: motorcycle_mAP@0.95_foggy: 0.250
[2025-09-09 07:40:46,838] INFO: bicycle_mAP@0.95_foggy: 0.263
[2025-09-09 07:40:46,869] INFO: [foggy] batch 28: mAP50 0.2809 (no-imp 6/10)


  0%|          | 0/663 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
direct_method.train()

In [ ]:
actmad.train()

In [ ]:
norm.train()

In [ ]:
dua.train()

In [ ]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat
from safetensors.torch import load_file

import utils

reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = 6

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 800, "width": 1280}
reference_preprocessor.do_resize = False

model_pretrained = RTDetrForObjectDetection(config=reference_config)
model_states = load_file("/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors", device="cpu")
model_pretrained.load_state_dict(model_states, strict=False)

for param in model_pretrained.parameters():
    param.requires_grad = False  # Freeze

# Initialize Model
model_pretrained

In [ ]:
model, optimizer = utils.add_adapters(model_pretrained, device = None, reduction_ratio=32, target_stages=[0, 1, 2, 3])

In [ ]:
for n,m in model.named_modules():
    if m.__class__.__name__ == "conv_task":
        print(n, hasattr(m, "down_proj"), hasattr(m, "up_proj"), getattr(m, "is_proj", None))
        params = [pn for pn,p in m.named_parameters() if p.requires_grad]
        print("trainable:", params)

In [ ]:
import torch
from collections import OrderedDict

path = "/workspace/ptta/faster_rcnn_r50_shift.pth"   # cfg.TEST.ADAPTATION.SOURCE_FEATS_PATH
s = torch.load(path, map_location="cpu")

# 1) 최상위 키 확인
print("[top-level keys]:", list(s.keys()))

# 2) kl_div 존재/타입 확인
kl = s.get("kl_div", None)
print("has kl_div?", kl is not None, "type:", type(kl))

# # 3) kl_div가 dict인 경우: 키/샘플 값 미리보기
# if isinstance(kl, dict):
#     print("kl_div keys (len={}):".format(len(kl)), list(kl.keys())[:10])
#     # 값이 텐서일 수 있어요 → 파이썬 숫자로 변환해 미리보기
#     sample_items = list(kl.items())[:10]
#     preview = {k: (v.item() if hasattr(v, "item") else float(v) if isinstance(v, (int,float)) else type(v).__name__)
#                for k, v in sample_items}
#     print("kl_div preview:", preview)

# # 4) 텐서/리스트 등 다른 형태 대비
# elif hasattr(kl, "shape"):  # torch.Tensor
#     print("kl_div tensor shape:", kl.shape)
#     print("first few:", kl.flatten()[:10])
# elif isinstance(kl, list):
#     print("kl_div list len:", len(kl), "first few:", kl[:10])

# # 5) 전체 합/통계 (dict[...]->scalar 값 가정)
# def to_scalar(x):
#     if hasattr(x, "item"):
#         return x.item()
#     return float(x)

# if isinstance(kl, dict):
#     vals = [to_scalar(v) for v in kl.values()]
#     print("sum:", sum(vals), "mean:", sum(vals)/len(vals) if vals else None)

In [ ]:
print(model.s_div.values())